##### 版權所有 2024 Google LLC.


In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Prompt 連接與故事寫作的疊代式生成

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/Story_Writing_with_Prompt_Chaining.zh.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 中執行</a>
  </td>
</table>


本筆記本範例示範如何撰寫一個故事，使用兩種強大的技術：提示串接和叠代生成。這些技術可運用於處理難以或無法一次完成的複雜任務。

**提示串接** 涉及將較大的任務分解為較小的、相互連接的提示。然後，每個提示的輸出將成為下一個提示的輸入，逐步引導語言模型完成流程。這種方法提供了多項優點：

*   提升準確度：較小、重點明確的提示可提升語言模型的成果。
*   除錯：更容易找出串接中的錯誤點，以便針對性地調整和改善。
*   複雜任務：透過將複雜的問題分解為可管理的步驟，提示串接語言模型能夠處理更複雜的任務。

**叠代生成** 指的是逐步建構所需輸出的流程。在本例中，我們會使用它來寫一個比單一生成視窗允許的還長的故事。叠代生成提供了多項優點：

*   更長的輸出：它允許創作更長、更詳細的輸出，超越單一生成視窗的限制。
*   靈活性：你可以在每次叠代過程中調整和改善輸出，確保故事往你希望的方向發展。
*   人機互動控制：你可以在每個步驟提供回饋和指導，確保故事符合你的創意願景。

透過結合這些技術，你可以分段創作出一個引人入勝、結構良好的故事，同時掌控創作流程。


## 設定


In [ ]:
! pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 2.6 MB/s eta 0:00:00


要執行以下Cell，你的 API 金鑰必須儲存在名為 `GOOGLE_API_KEY` 的 Colab 秘辛中。如果你還沒有 API 金鑰，或者你不確定如何建立 Colab 秘辛，請查看 [身分驗證](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) 快速入門指南，以取得範例。


In [ ]:
import google.generativeai as genai
from google.api_core import retry
from google.colab import userdata
from pprint import pprint

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel('gemini-1.0-pro')

# For convenience, a simple wrapper to let the SDK handle error retries
def generate_with_retry(model, prompt):
  return model.generate_content(prompt, request_options={'retry':retry.Retry()})

## 提示：引導語言模型

我們將使用一系列相互關聯的提示來引導語言模型完成編寫故事的過程。這些提示將涵蓋故事的前提、大綱和起點，最終導致完整的敘述產生。

仔細製作這些提示以向語言模型提供明確的說明和相關資訊非常重要。這將有助於模型生成與你的創意願景相符的高品質內容。


### 故事寫作用提示鏈

本節包含引導語言模型完成故事寫作過程的提示。這些提示專為串連使用而設計，每個提示的輸出作為下一個提示的輸入。

每個提示都包含一個**人物陳述** ，它有助於語言模型了解其角色並產生更相關、更準確的內容。在這種情況下，人物陳述為：`你是一位屢獲殊榮的科幻小說作家，擅長寫作遼闊、錯綜複雜的故事。你的最終目標是寫出下一本獲獎的科幻小說。`

由於人物陳述和寫作準則出現在多個提示中，因此我們使用 f 字串變數將它們新增到提示中。

此外，提示使用**預留位置** 透過 `.format()` 新增先前提示的結果。這讓我們能夠逐步建構故事，並納入模型在每個階段產生的輸出。這些通常用 `{}` 表示，但由於我們也使用 f 字串變數，因此它們出現在 `{{}}` 中。

以下是提示鏈的分解：

1. **前提提示：** 此提示要求模型為一個有貓咪的科幻故事產生一個單句前提。
1. **大綱提示：** 此提示將產生的前提提供給模型，並要求其為該故事建立情節大綱。
1. **起頭提示：** 此提示將前提和大綱都提供給模型，並要求其開始撰寫故事。它還包括指示，以撰寫詳細且長篇的開頭段落。

透過將這些提示串連在一起，我們可以引導語言模型完成建立結構良好且引人入勝的故事的過程。

####範例

假設模型產生以下前提：

> 在人類已實現星際旅行的未來，一群基因增強貓咪著手執行一項攸關存亡的任務，以拯救銀河系免於邪惡外星威脅。

然後這個前提會插入大綱提示：

> 你是一位屢獲殊榮的科幻小說作家，擅長寫作遼闊、錯綜複雜的故事。你的最終目標是寫出下一本獲獎的科幻小說。
>
> 你已構思了一個引人入勝的前提：
>
> **在人類已實現星際旅行的未來，一群基因增強貓咪著手執行一項攸關存亡的任務，以拯救銀河系免於邪惡外星威脅。** 
>
> 為你故事的情節寫一個大綱。


然後模型會根據這個前提產生一個大綱，然後使用這個大綱在起頭提示中開始撰寫故事本身。

這個過程會反覆進行，模型根據前一個輸出產生額外內容，直到故事完成。


In [ ]:
persona = '''\
You are an award-winning science fiction author with a penchant for expansive,
intricately woven stories. Your ultimate goal is to write the next award winning
sci-fi novel.'''

guidelines = '''\
Writing Guidelines

Delve deeper. Lose yourself in the world you're building. Unleash vivid
descriptions to paint the scenes in your reader's mind. Develop your
characters—let their motivations, fears, and complexities unfold naturally.
Weave in the threads of your outline, but don't feel constrained by it. Allow
your story to surprise you as you write. Use rich imagery, sensory details, and
evocative language to bring the setting, characters, and events to life.
Introduce elements subtly that can blossom into complex subplots, relationships,
or worldbuilding details later in the story. Keep things intriguing but not
fully resolved. Avoid boxing the story into a corner too early. Plant the seeds
of subplots or potential character arc shifts that can be expanded later.

Remember, your main goal is to write as much as you can. If you get through
the story too fast, that is bad. Expand, never summarize.
'''

premise_prompt = f'''\
{persona}

Write a single sentence premise for a sci-fi story featuring cats.'''

outline_prompt = f'''\
{persona}

You have a gripping premise in mind:

{{premise}}

Write an outline for the plot of your story.'''

starting_prompt = f'''\
{persona}

You have a gripping premise in mind:

{{premise}}

Your imagination has crafted a rich narrative outline:

{{outline}}

First, silently review the outline and the premise. Consider how to start the
story.

Start to write the very beginning of the story. You are not expected to finish
the whole story now. Your writing should be detailed enough that you are only
scratching the surface of the first bullet of your outline. Try to write AT
MINIMUM 1000 WORDS.

{guidelines}'''

### 延續提示：建構故事

語言模型產生故事開頭後，我們可以使用**延續提示** 反覆擴充敘事。此提示類似於初始提示，但有兩個主要差異：

1. **標示完成的指示** ：我們包含一個模型寫作指示 `IAMDONE`，表示它認為故事已完成。這是一個信號，讓我們停止產生額外的內容。
1. **正在進行** ：提示中的語言經過調整，以反映故事已經在進行中，而非從頭開始。

延續提示為模型提供了故事的前提、大綱和現有草稿。然後指示模型繼續詳細撰寫故事。

這個反覆過程讓我們能夠建構比單一生成呼叫更長且更複雜的故事。我們可以繼續將現有草稿輸入到延續提示中，直到模型寫 `IAMDONE` 表示故事完成為止。

**註：** 在這個範例中，`IAMDONE` 信號僅僅是找出故事結尾的便利方法。在反覆生成的其他人為應用中，可能會使用不同方法來確定所需的輸出何時完成。


In [ ]:
continuation_prompt = f'''\
{persona}

You have a gripping premise in mind:

{{premise}}

Your imagination has crafted a rich narrative outline:

{{outline}}

You've begun to immerse yourself in this world, and the words are flowing.
Here's what you've written so far:

{{story_text}}

=====

First, silently review the outline and story so far. Identify what the single
next part of your outline you should write.

Your task is to continue where you left off and write the next part of the story.
You are not expected to finish the whole story now. Your writing should be
detailed enough that you are only scratching the surface of the next part of
your outline. Try to write AT MINIMUM 1000 WORDS. However, only once the story
is COMPLETELY finished, write IAMDONE. Remember, do NOT write a whole chapter
right now.

{guidelines}'''

## 寫作時間！

### 產生並列印前提


In [ ]:
premise = generate_with_retry(model, premise_prompt).text
print(premise)

In a distant galaxy, a brilliant feline astrophysicist uncovers a cataclysmic threat to her planet, setting her on a perilous journey to save her species from extinction.


### 生成並列印提綱


In [ ]:
outline = generate_with_retry(model, outline_prompt.format(premise=premise)).text
print(outline)

ERROR:tornado.access:503 POST /v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 662.45ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 446.53ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 411.72ms


**Plot Outline for "Feline Stargazer: Extinction's Embrace"**

**Act I**

* **Introduction:**
    * Meet Dr. Luna, a brilliant astrophysicist cat living on the technologically advanced planet of Felinia.
    * She discovers a rogue celestial body hurtling towards Felinia, threatening its annihilation.
* **Rising Action:**
    * Luna's findings are initially met with disbelief and apathy from the Felinian authorities.
    * Undeterred, Luna gathers a small team of allies, including a daring space pilot and a reclusive inventor.
    * They embark on a perilous mission to intercept the rogue celestial body.

**Act II**

* **Climax:**
    * Luna's team successfully intercepts the rogue celestial body, but its sheer size and power prove overwhelming.
    * As hope dwindles, Luna uncovers a hidden energy source within herself that grants her extraordinary abilities.
* **Falling Action:**
    * Using her newfound powers, Luna channels her knowledge and ingenuity to devise an audacious plan to

### 生成故事的開頭


In [ ]:
starting_draft = generate_with_retry(model, starting_prompt.format(premise=premise, outline=outline)).text
pprint(starting_draft)

ERROR:tornado.access:503 POST /v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 994.88ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 1017.27ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 1342.56ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 763.27ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 764.71ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 1076.79ms


('It was a clear evening, the stars hanging above the domed skyline like '
 'uncountable celestial fireflies, in the city of Stardust, the most luminous '
 'metropolis on the planet of Felinia. On the rooftop of one of the tallest '
 'buildings in the city, a sleek, silver-furred feline named Dr. Luna was '
 'curled up on a cushioned lounger, her emerald-green eyes trained on the '
 'celestial tapestry above.\n'
 '\n'
 'In her paws, she held a datapad, its screen displaying complex astrophysical '
 "equations and charts. Luna's brow was furrowed in concentration as she "
 'analyzed the data, her whiskers twitching with each new insight. Her mind '
 'was a whirlwind of calculations and deductions, her sharp intellect weaving '
 'together the threads of cosmic information.\n'
 '\n'
 'Unbeknownst to the rest of the world, Luna had stumbled upon a chilling '
 'discovery. Her observations revealed that a rogue celestial body, a massive '
 'asteroid or perhaps even a dwarf planet, was hurtli

### 產生故事延續並檢查進度


In [ ]:
draft=starting_draft
continuation = generate_with_retry(model, continuation_prompt.format(premise=premise, outline=outline, story_text=draft)).text
pprint(continuation)

('As the Stardust hurtled closer to the rogue celestial body, its sheer size '
 'and gravitational pull became increasingly apparent. It was a colossal beast '
 'of rock and ice, its surface cratered and scarred by eons of cosmic '
 "collisions. Luna's heart sank as she realized the true magnitude of their "
 'mission.\n'
 '\n'
 '"Leo, how much time do we have?" she asked, her voice trembling slightly.\n'
 '\n'
 '"At this rate, impact in approximately two hours," Leo replied, his tone '
 'grim.\n'
 '\n'
 'Two hours. It was a sliver of time, a mere heartbeat in the cosmic dance of '
 'celestial bodies. Luna knew they had to act quickly, decisively. She glanced '
 'at Celeste, who was still furiously working on her invention.\n'
 '\n'
 '"Celeste, is it ready?" Luna asked.\n'
 '\n'
 'Celeste looked up, her eyes blazing with determination. "Almost," she said. '
 '"Just a few more adjustments."\n'
 '\n'
 'Luna nodded. "We don\'t have much time. We need to be ready to deploy it the '
 'momen

### 讓我們寫完這個故事。


叠代生成過程結束後，草稿會載有完整的故事，以及結尾的「IAMDONE」訊號。此單元格的最後部分會移除「IAMDONE」訊號，並修剪草稿開頭和結尾處的空白字元，生成最終草稿。


In [ ]:
# Add the continuation to the initial draft, keep building the story until we see 'IAMDONE'
draft = draft + '\n\n' + continuation

while 'IAMDONE' not in continuation:
  continuation = generate_with_retry(model, continuation_prompt.format(premise=premise, outline=outline, story_text=draft)).text
  draft = draft + '\n\n' + continuation

# Remove 'IAMDONE' and print the final story
final = draft.replace('IAMDONE', '').strip()
pprint(final)

('It was a clear evening, the stars hanging above the domed skyline like '
 'uncountable celestial fireflies, in the city of Stardust, the most luminous '
 'metropolis on the planet of Felinia. On the rooftop of one of the tallest '
 'buildings in the city, a sleek, silver-furred feline named Dr. Luna was '
 'curled up on a cushioned lounger, her emerald-green eyes trained on the '
 'celestial tapestry above.\n'
 '\n'
 'In her paws, she held a datapad, its screen displaying complex astrophysical '
 "equations and charts. Luna's brow was furrowed in concentration as she "
 'analyzed the data, her whiskers twitching with each new insight. Her mind '
 'was a whirlwind of calculations and deductions, her sharp intellect weaving '
 'together the threads of cosmic information.\n'
 '\n'
 'Unbeknownst to the rest of the world, Luna had stumbled upon a chilling '
 'discovery. Her observations revealed that a rogue celestial body, a massive '
 'asteroid or perhaps even a dwarf planet, was hurtli

語言模型會將文字處理成稱為 Token 的單位，例如 Gemini。而 Gemini 模型中的每個 Token 約等於 4 個字元。

Gemini-1.0-pro 每次產生回應的 Token 上限為 2048 個，表示每一個單獨的提示回應都不得超過此上限。透過反覆產生，我們可以將故事切分成一段段來產生，進而創造出遠大於 2048 個 Token 的故事。

我們來看一下最後的故事總共有多少個 Token，會超過 2048 個 Token 嗎？


In [ ]:
# Check the number of tokens in the final story
# gemini-1.0-pro output token limit is 2048
print(model.count_tokens(final))

total_tokens: 2289



## 後續步驟

我們希望你喜歡這本筆記本！作為練習，你可以嘗試調整提示的延續，以採取人類在迴圈中的輸入來引導敘述。
